In [11]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import spacy
import nltk
import requests
import certifi
import time
from nltk.corpus import brown
nltk.download('brown')

nlp = spacy.load("en_core_web_sm")
sw_spacy = nlp.Defaults.stop_words

from nltk.stem.porter import *
from collections import defaultdict

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Harsha\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [12]:
vocab = set(brown.words())

stemmer = PorterStemmer()
d = defaultdict(set)
for v in vocab:
    d[stemmer.stem(v)].add(v)  

In [13]:
possible = d[stemmer.stem('letting')]
for p in possible:
    if p.islower():
        print(p)

let
letting
lets


In [14]:
def check_noun(word):
    doc = nlp(word)

    if(doc[0].tag_ == 'NNP'):
        return True
    else:
        return False

check_noun("chakri")

True

In [15]:
test = ['free', 'will', 'changing', 'destiny', 'choosing', 'love', 'choice', 'soul', 'conditioning', 'acceptance']

if 'soul' in test:
    print('works')

works


In [16]:
def remove_stopwords(text):

    # words = [word for word in text.split() if word.lower() not in sw_spacy]
    # new_text = " ".join(words)
    # print(new_text)
    # print("Old length: ", len(text))
    # print("New length: ", len(new_text))

    res = []
    for word in text:
        if word not in sw_spacy:
            res.append(word)
    return res


In [17]:


page = urlopen(
    "https://ezinearticles.com/?Live-In-the-Present-to-Create-the-Life-You-Desire&id=6504152",
    context=ssl.create_default_context(cafile=certifi.where()))


soup = BeautifulSoup(page, 'html.parser')
title = soup.title.string.lower().split()
key = soup.select("meta[name='keywords']")[0]['content'].lower().split(",")

print(title)
keywords = [] 
for k in key:
    
    for word in k.split():
        if word not in keywords:
            keywords.append(word)

result=[]

print(keywords)

for word in title:
    if check_noun(word):
        if word not in result:
            result.append(word)

for word in keywords:
    possible = d[stemmer.stem(word)]
    for p in possible:
        if p in title:
            if p not in result:
                result.append(p)
        
result= remove_stopwords(result)
print(result)

['live', 'in', 'the', 'present', 'to', 'create', 'the', 'life', 'you', 'desire']
['the', 'present', 'living', 'in', 'now', 'gift', 'of', 'moment', 'unknown', 'future', 'painful', 'past']
['present', 'live']


In [18]:
url="https://ezinearticles.com/?Live-In-the-Present-to-Create-the-Life-You-Desire&id=6504152"
requests.get(url,verify=True)

<Response [200]>

In [19]:
page = urlopen(
    "https://ezinearticles.com/?Live-In-the-Present-to-Create-the-Life-You-Desire&id=6504152",
    context=ssl.create_default_context(cafile=certifi.where()))


soup = BeautifulSoup(page, 'html.parser')
name = 'ezinearticles'
found = []
for a in soup.select('a[href]'):
    b = a['href']
    if 'https://' + name + '.com' in b and b not in found:
        found.append(b)
        break

print(found)




[]


In [20]:
def full_url(partial_url):
    return "https://ezinearticles.com"+partial_url

In [21]:
mydivs = soup.find_all("ul", {"class": "ea-similar-articles ea-article-list"})[0]
type(mydivs)
#mydivs
test1 = mydivs.findChildren('a')
for i in range(0,len(test1)):
    tag = test[i]
    print(full_url(tag.get_attribute_list('href')[0]))


AttributeError: 'str' object has no attribute 'get_attribute_list'

In [22]:
type(test[0].get_attribute_list('href')[0])

AttributeError: 'str' object has no attribute 'get_attribute_list'

In [23]:

def spider(found_titles, url, found):
    try:
        page = urlopen(url,context=ssl.create_default_context(cafile=certifi.where()))
        soup = BeautifulSoup(page, 'html.parser')

        og_title = soup.title.string.lower()
        title = soup.title.string.lower().split()
        key = soup.select("meta[name='keywords']")[0]['content'].lower().split(",")
        keywords = [] 
        for k in key:
            for word in k.split():
                if word not in keywords:
                    keywords.append(word)

        result=[]

        print(keywords)

        for word in title:
            if check_noun(word):
                if word not in result:
                    result.append(word)

        for word in keywords:
            possible = d[stemmer.stem(word)]
            for p in possible:
                if p in title:
                    if p not in result:
                        result.append(p)
        result= remove_stopwords(result)
        
        if len(result) > 0 and og_title not in found_titles:
            found_titles.append(og_title)
            print(og_title)
            print(result)

            f = open('test.txt', 'a')
            f.write(
                og_title + "\t" + ' '.join(
                    k.replace(' ', '_') for k in result
                ) + "\n"
            )
            f.close()
        
        mydivs = soup.find_all("ul", {"class": "ea-similar-articles ea-article-list"})[0]
        #print(mydivs)
        
        if mydivs:
            test1 = mydivs.findChildren('a')
            #print("here: ",test1)
            if test1:
                for i in range(0,len(test1)):
                    tag = test1[i]
                    
                    url_here = full_url(tag.get_attribute_list('href')[0])
                    print("url here: ",url_here)
                    if url_here not in found:
                        found.append(url_here)
                        time.sleep(30)
                        spider(found_titles,url_here,found)
        
    except:
        pass

In [ ]:
start_url = [
    "https://ezinearticles.com/?Some-Dangers-From-Pandemic-Fatigue&id=10554149",
    "https://ezinearticles.com/?Uses-of-Silicon-Carbide-Ceramics-in-Different-Industries&id=10550034",
    "https://ezinearticles.com/?What-Might-Be-Next-In-The-Economy?&id=10554161",
    "https://ezinearticles.com/?Appointments---Manage-Your-Time-Better-At-Home-to-Be-Effective&id=10553717",
    "https://ezinearticles.com/?Appointments---The-Importance-of-Making-Appointments-for-Entrepreneurs&id=1055409"
    ]

found = [start_url[0]]
found_titles = []
for starter in start_url:
    spider(found_titles,start_url[0],found)

['public', 'health', 'pandemic', 'fatigue', 'mandates', 'common', 'sense', 'vaccines', 'masks']
some dangers from pandemic fatigue
['pandemic', 'fatigue']
url here:  https://ezinearticles.com/?5-Common-Sense-Reasons,-To-Remain-Vigilant,-About-Public-Health-Issues&id=10308823
['public', 'health', 'common', 'sense', 'social', 'distancing', 'wear', 'a', 'mask', 'pandemic', 'virus', 'reopening', 'america', 'trump']
5 common sense reasons, to remain vigilant, about public health issues
['public', 'health', 'common', 'sense']
url here:  https://ezinearticles.com/?Use-Common-Sense-And-Social-Responsibility,-Or-Delay-Pandemic-Recovery!:-5-Considerations&id=10325977
['common', 'sense', 'social', 'responsibility', 'public', 'health', 'pandemic', 'recovery', 'wear', 'a', 'mask', 'distancing']
use common sense and social responsibility, or delay pandemic recovery!: 5 considerations
['common', 'sense', 'social', 'pandemic']
url here:  https://ezinearticles.com/?6-Basic-Steps-To-Gain-Control-Over-Th

In [ ]:
found

['https://ezinearticles.com/?Uses-of-Silicon-Carbide-Ceramics-in-Different-Industries&id=10550034',
 'https://ezinearticles.com/?All-About-Silicon-Carbide-Ceramic&id=9875605',
 'https://ezinearticles.com/?Know-About-All-the-Properties-of-Silicon-Carbide-Ceramic&id=9649701',
 'https://ezinearticles.com/?Low-Cost-Synthesis-of-Silicon-Carbide-and-Silicon-Nitride&id=7087791',
 'https://ezinearticles.com/?Reaction-Bonded-Silicon-Carbide:-Advantages,-Application-and-Products&id=10543934',
 'https://ezinearticles.com/?Know-the-Uses-of-Reaction-Bonded-Silicon-Carbide&id=9911315',
 'https://ezinearticles.com/?Some-Useful-Facts-of-Silicon-Carbide&id=9616116',
 'https://ezinearticles.com/?Know-the-Benefits-of-Using-Silicon-Carbide&id=9590883',
 'https://ezinearticles.com/?Know-The-Useful-Benefits-Of-Using-Silicon-Carbide&id=9664080',
 'https://ezinearticles.com/?Some-Important-Features-of-Silicon-Carbide&id=9787995',
 'https://ezinearticles.com/?Know-the-Several-Applications-of-Silicon-Carbide&id

In [24]:
ted_url = "https://www.ted.com/talks/david_keith_a_critical_look_at_geoengineering_against_climate_change"

In [25]:
def nospecial(text):
	import re
	text = re.sub("[^a-zA-Z0-9]+", "",text)
	return text

In [26]:
import nest_asyncio
nest_asyncio.apply()


from requests_html import HTMLSession

session = HTMLSession()
r = session.get(ted_url)
r.html.arender()
soup = BeautifulSoup(r.text, 'html.parser')
test = soup.find_all("div",{"class":"tabs"})
divs = soup.find_all("div",{"id":"tabs--1--panel--0"})
test


<ipython-input-26-8466ba985218>:9: RuntimeWarning: coroutine 'HTML.arender' was never awaited
  r.html.arender()


[]

[]

In [27]:
import requests
custom_User_agent = "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"
url = ted_url
response = requests.get(url, headers={"User-Agent": custom_User_agent})
html_text = response.text
soup = BeautifulSoup(html_text, 'html.parser')

In [ ]:

test = soup.find_all("div",{"class":"tabs"})
test = soup.find_all("div",{"class":"pt-6"})
divs = soup.find_all("div",{"id":"tabs--1--panel--0"})

print(test,divs)

In [29]:
# import nest_asyncio
# nest_asyncio.apply()

# from requests_html import AsyncHTMLSession

# session = AsyncHTMLSession()
# r = session.get(ted_url)
# r.html.render()

page = urlopen(ted_url,context=ssl.create_default_context(cafile=certifi.where()))

soup = BeautifulSoup(page, 'html.parser')
#soup = BeautifulSoup(page, 'lxml')
og_title = soup.title.string.lower()
title = soup.title.string.lower().split()
key = soup.select("meta[name='keywords']")[0]['content'].lower().split(",")
keywords = [] 
for k in key:
    for word in k.split():
        if word not in keywords:
            keywords.append(word)

result=[]


for i in range(0,len(title)):
    title[i] = nospecial(title[i])

while("" in title):
    title.remove("")

# print(title)
# print(keywords)

for word in title:
    if check_noun(word) :
        if word not in result:
            result.append(word)

for word in title:
    if word in keywords:
        if word not in result:
            result.append(word)

for word in keywords :
    possible = d[stemmer.stem(word)]
    possible = [p.lower() for p in possible]
    for p in possible:
        if p in title:
            if p not in result:
                result.append(p)

result= remove_stopwords(result)
if "ted" in result:
    result.remove("ted")
if "talk" in result:
    result.remove("talk")
if "talks" in result:
    result.remove("talks")
# print(result)
# for a in soup.find_all('a'):
#     print(a)
# for x in soup.find_all('aside'):
#     print(x)
aside = soup.find_all('aside')
hyper = soup.find_all('a')
# for i in soup.find_all('aside'):
#     print(i.text)
# images = []
# for img in soup.findAll('img'):
#     images.append(img.get('src'))

# print(images)
# mydivs = soup.find_all("a", {"class": "mb-5 block p-3"})
# print(mydivs)



In [30]:
type(aside[0])

bs4.element.Tag

In [31]:
aside[0].findChildren('a')

[<a class="rounded-sm" href="/membership?utm_medium=display&amp;utm_source=talkpage&amp;utm_campaign=membership-ted&amp;utm_content=06172021-talkpagebanner"><div class="py-3 relative rounded-sm inline-flex items-center select-none transition-colors duration-1000 justify-center min-w-button px-4 min-h-button-sm text-white bg-red-700 hover:bg-red-900"><div class="transition-opacity duration-300 inline-flex items-center opacity-100"><div class="text-xs font-bold">Join now</div></div></div></a>,
 <a class="inline-block py-1 text-tui-sm capitalize underline" href="/topics/climate+change">climate change</a>,
 <a class="inline-block py-1 text-tui-sm capitalize underline" href="/topics/global+issues">global issues</a>,
 <a class="inline-block py-1 text-tui-sm capitalize underline" href="/topics/science">science</a>,
 <a class="inline-block py-1 text-tui-sm capitalize underline" href="/topics/technology">technology</a>,
 <a class="inline-block py-1 text-tui-sm capitalize underline" href="/topic

In [ ]:
hyper

In [ ]:
mydivs = soup.find_all("div", {"class": "relative w-full overflow-hidden tabs-overflow-container"})
mydivs

[<div class="relative w-full overflow-hidden tabs-overflow-container"><span aria-hidden="true" class="absolute top-0 h-full w-14 transition-opacity -mt-px from-white pointer-events-none z-10 left-0 bg-gradient-to-r -ml-px opacity-0"></span><div class="no-scrollbar-horizontal flex overflow-auto"><div aria-orientation="horizontal" class="flex flex-nowrap w-full" data-reach-tab-list="" role="tablist"><button aria-controls="tabs--panel---1" aria-selected="false" class="group outline-inside text-gray-500 hover:text-gray-700 focus:text-gray-700 transition-colors pr-3 pl-3 first:pl-0 last:pr-0 relative md:pr-4 md:pl-4 xl:pr-5 xl:pl-5" data-orientation="horizontal" data-reach-tab="" id="tabs--tab---1" role="tab" tabindex="-1" type="button"><div class="relative tab-text pb-3"><h4 class="text-lg xl:text-2xl font-bold tracking-normal whitespace-nowrap out">Watch next</h4></div></button></div></div><span aria-hidden="true" class="absolute top-0 h-full w-14 transition-opacity -mt-px from-white poin

In [32]:
mydivs = soup.find_all("div", {"id": "tabspanel"})
mydivs

[]

In [33]:
soup.findChildren('a')[2]

<a class="outline-inside flex items-center py-4" href="/"><svg class="h-6 lg:mr-3" fill="none" viewbox="0 0 68 24" xmlns="http://www.w3.org/2000/svg"><path clip-rule="evenodd" d="M21.419 0v6.151h-6.763V24h-7.44V6.151H.453V0H21.42zm21.484 0v6.141l-12.918.01v2.946h12.918v5.73l-12.918-.009v3.03h12.918V24H22.546V0h20.357zm13.446 0c8.048 0 10.889 5.916 10.889 11.966C67.237 19.328 63.314 24 54.894 24H44.142V0h12.207zm-2.4 6.151H51.58V17.85h2.908c4.633 0 5.31-3.731 5.31-5.983 0-1.513-.474-5.715-5.85-5.715z" fill="#EC1015" fill-rule="evenodd"></path></svg><div class="text-sm hidden text-gray-500 lg:block">Ideas worth spreading</div></a>

In [36]:

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time



driver = webdriver.Chrome('./personalTests/chromedriver.exe') 
driver.get(ted_url) 
time.sleep(5) 
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
driver.close() 

In [ ]:
divs =soup.find_all("a",{"class":"mb-5 block p-3"})
divs

In [53]:
for d in divs:
    href = d.get_attribute_list("href")
    print(href[0])
    

/talks/juan_enriquez_using_biology_to_rethink_the_energy_challenge
/talks/john_doerr_salvation_and_profit_in_greentech
/talks/alex_steffen_the_route_to_a_sustainable_future
/talks/james_hansen_why_i_must_speak_out_about_climate_change
/talks/gavin_schmidt_the_emergent_patterns_of_climate_change
/talks/al_gore_new_thinking_on_the_climate_crisis


In [ ]:
page = urlopen(ted_url,context=ssl.create_default_context(cafile=certifi.where()))

soup = BeautifulSoup(page, 'html.parser')

In [67]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

start_url = "https://www.ted.com/talks/olivia_vinckier_a_colorful_case_for_outside_the_box_thinking_on_identity"

driver = webdriver.Chrome('./personalTests/chromedriver.exe') 


def spider(url,titles,found):
    

    driver.get(url) 
    timeout = 3
    try:
        element_present = EC.presence_of_element_located((By.ID, 'main'))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
    finally:
        print("Page loaded")
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    og_title = soup.title.string.lower()
    title = soup.title.string.lower().split()
    key = soup.select("meta[name='keywords']")[0]['content'].lower().split(",")
    keywords = [] 
    for k in key:
        for word in k.split():
            if word not in keywords:
                keywords.append(word)

    result=[]


    for i in range(0,len(title)):
        title[i] = nospecial(title[i])

    while("" in title):
        title.remove("")

    # print(title)
    # print(keywords)

    for word in title:
        if check_noun(word) :
            if word not in result:
                result.append(word)

    for word in title:
        if word in keywords:
            if word not in result:
                result.append(word)

    for word in keywords :
        try:
            possible = d[stemmer.stem(word)]
            possible = [p.lower() for p in possible]
            for p in possible:
                if p in title:
                    if p not in result:
                        result.append(p)
        except:
            pass

    result= remove_stopwords(result)
    if "ted" in result:
        result.remove("ted")
    if "talk" in result:
        result.remove("talk")
    if "talks" in result:
        result.remove("talks")
    if len(result)>0 and og_title not in titles:
        titles.append(og_title)
    f = open('test.txt', 'a')
    f.write( og_title + "\t" + ' '.join(k.replace(' ', '_') for k in result) + "\n")
    f.close()
    divs =soup.find_all("a",{"class":"mb-5 block p-3"})
    base_url = "https://www.ted.com"
    for d in divs:
        href = d.get_attribute_list("href")
        if base_url+href[0] not in found:
            found.append(base_url+href[0])
            spider(base_url+href[0],titles,found)


spider(start_url,[],[start_url])
driver.close() 

Timed out waiting for page to load
Page loaded
Timed out waiting for page to load
Page loaded
Timed out waiting for page to load
Page loaded
Timed out waiting for page to load
Page loaded
Timed out waiting for page to load
Page loaded
Timed out waiting for page to load
Page loaded
Timed out waiting for page to load
Page loaded


In [62]:
from requests_html import HTMLSession
session = HTMLSession()
response = session.get(start_url)
response.html.arender()



<coroutine object HTML.arender at 0x000001E994BCADC0>

In [64]:
response.html.find("a")

[<Element 'a' class=('text-base', 'leading-md', 'sr-only', 'focus:not-sr-only', 'font-bold', 'outline-inside', 'flex', 'items-center', 'justify-center', 'bg-white', 'focus:absolute', 'focus:h-full', 'focus:w-full') href='#maincontent'>,
 <Element 'a' class=('text-base', 'leading-md', 'sr-only', 'focus:not-sr-only', 'font-bold', 'outline-inside', 'flex', 'items-center', 'justify-center', 'bg-white', 'focus:absolute', 'focus:h-full', 'focus:w-full') href='#navigation-search'>,
 <Element 'a' class=('outline-inside', 'flex', 'items-center', 'py-4') href='/'>,
 <Element 'a' role='menuitem' tabindex='-1' data-reach-menu-link='' data-selected='' data-valuetext='' data-reach-menu-item='' class=('menuItem', 'px-8', 'py-4', 'first:pt-8', 'last:pb-8', 'block', 'hover:bg-gray-50') href='/talks'>,
 <Element 'a' role='menuitem' tabindex='-1' data-reach-menu-link='' data-selected='' data-valuetext='' data-reach-menu-item='' class=('menuItem', 'px-8', 'py-4', 'first:pt-8', 'last:pb-8', 'block', 'hover